This notebook is an exact replica of the trajectory builder in terms of functionality. The difference is that this notebook is used to create a visualization of the trajectories you wish to create and demonstrate how the notebook works. Nothing is saved to file in this notebook. You can skip the 1st cell as it only imports the libraries and defines the functions, all of which are described in the other notebook.

In [ ]:
import sys, subprocess,pkg_resources
required = {'numpy','scipy','matplotlib','ipython'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import special
from IPython.display import HTML

import matplotlib
#parameters that allow animations larger that 20 Mb
matplotlib.rcParams['animation.embed_limit'] = 2**128

def t_dist(tau,t,t_mid):
    #distributes the angle as a function of time from pi to 0
    delta_t = t - t_mid
    return np.pi*(1 - special.erf(delta_t/tau))/2



def var_calc(t9,nn):
    #calculates the variables required to parametrize the trajectory
    A= np.array([[-1,0,1],[1,0,1],[0,-1,1]])
    
    x = np.linalg.solve(A,t9)
    rx_cos = x[0]
    ry_sin = x[1]
    x_0 = x[2]

    x = np.linalg.solve(A,nn)
    rx_sin = x[0]
    ry_cos = x[1]
    y_0 = x[2]

    return [x_0,y_0,rx_cos,ry_sin,rx_sin,ry_cos]

def pulse(varia,t,param_list,time_func):
    time = np.linspace(t[0],t[1],param_list['pulse_size'])

    theta = time_func(param_list['tau'],time,t[1]/2)

    x_t = varia[2]*np.cos(theta) - varia[3]*np.sin(theta) + varia[0]
    y_t = varia[4]*np.cos(theta) - varia[5]*np.sin(theta) + varia[1]
        
    return x_t,y_t,time
    
def pulse_return(varia,t, param_list, t9,nn, time_func,t9_return):

    t_pulse = param_list['pulse_frac']*t[1]
    t_return_start = t_pulse*(1 + 1e-3)
    
    slope = (nn[1] - nn[0])/(t9[1] - t9[0])
    inter = nn[1] - slope*t9[1]

    x_t,y_t,time = pulse(varia,[t[0],t_pulse],param_list,time_func)

    time = np.append(time,np.linspace(t_return_start,t[1],param_list['return_size']))
    x_t = np.append(x_t,np.linspace(t9[1],t9_return,param_list['return_size']))
    y_t = np.append(y_t,slope*np.linspace(t9[1],t9_return,param_list['return_size']) + inter)
    
    return x_t,y_t,time

def pulse_return_loop(varia,t,param_list,t9,nn,time_func):

    slope_vert = (nn[2] - nn[0])/(t9[2] - t9[0])
    slope_hori = (nn[1] - nn[0])/(t9[1] - t9[0])

    inter_hori = nn[0] - slope_hori*t9[0]

    dist_vert = ((nn[2] - nn[0])**2 + (t9[2] - t9[0])**2)**(1/2)
    dist_hori = ((nn[1] - nn[0])**2 + (t9[1] - t9[0])**2)**(1/2)

    t_loop = t[1]/param_list['n_loops']
    t9_return = t9[0] + param_list['return_scale']*dist_hori/(1 + slope_hori**2)**(1/2)
    x_t,y_t,time = pulse_return(varia,[t[0],t_loop],param_list,t9,nn,time_func,t9_return)
    

    for i in range(1, param_list['n_loops']):
        
        
        t9_1 = t9_return
        t9_2 = t9_return + dist_hori/(1 + slope_hori**2)**(1/2)
        t9_3 = t9_1 + (1 + param_list['peak_scale'])**(i) * dist_vert/(1 + slope_vert**2)**(1/2)

        t9_return = t9_1 + param_list['return_scale']*dist_hori/(1 + slope_hori**2)**(1/2)

        nn_1 = slope_hori*t9_1 + inter_hori
        nn_2 = slope_hori*t9_2 + inter_hori

        
        b = nn_1 - slope_vert*t9_1
        nn_3 = slope_vert*t9_3 + b

        varia = var_calc([t9_1,t9_2,t9_3],[nn_1,nn_2,nn_3])
        x_ti,y_ti,time_i = pulse_return(varia,[t[0],t_loop],param_list,[t9_1,t9_2,t9_3],[nn_1,nn_2,nn_3],time_func,t9_return)
        
        time = np.append(time, i*t_loop + time_i)
        x_t = np.append(x_t,x_ti)
        y_t = np.append(y_t,y_ti)
        
    return x_t,y_t,time

def traj_builder(t,t9,nn,traj,param_list,time_func = t_dist):
    #builds the trajectories and saves to file
    #t = 2x1 array of start and end time
    #t9 = 3x1 array containing start, end, and peak t9 (in that order)
    #nn = 3x1 array containing the power of 10 of the start, end, and peak of neutron number density (in that order)
    
    varia = var_calc(t9,nn)
    
    if traj == 'pulse':
        x_t,y_t,time = pulse(varia, t, param_list,time_func)
        
    elif traj == 'pulse_return':
        x_t, y_t,time = pulse_return(varia,t,param_list,t9,nn,time_func,t9[0])

    elif traj == 'pulse_return_loop':
        x_t,y_t,time = pulse_return_loop(varia,t,param_list,t9,nn,time_func)

    return x_t,y_t,time

def param_builder(var_list):
    param = {}
    for var_name in var_list:
        if var_name in globals():
            param[var_name] = globals()[var_name]
    return param

Here you can set all the parameters of the trajectory, and fine tune it to your liking

In [ ]:
#setup the parameters to build your trajectory

#temperature in billions of kelvin
t9_1 = 0.125 #start temperature
t9_2 = 0.25  #end temperature
t9_3 = 0.3   #temperature at peak n_n
####################################

#the power of 10 of neutron number density
nn_1 = 1.1  #start
nn_2 = 2    #end
nn_3 = 9    #peak
###########################################

#time
t_start = 0 #start time default time 0
t_end = 3.15e+12 #end time of the whole trajectory in seconds

#parameters for the trajectory
return_size = 15 #number of points in time for the linear return part of the trajectory
              #defaults to 15
pulse_frac = 0.95 #the amount of time per cycle allocated for the pulse 
             #defaults to 0.95
tau = 1e+12 #relaxation timescale for the error function, typically same order as t_end
pulse_size = 200 #total number of points in the pulse
             #defaults to 50
n_loops = 5 #total number of loops in the trajectory
return_scale = 0.5 #how far from t9_1 should the new loop start. 0 returns to it
                #defaults to 0
peak_scale = 0 #scales the next pulse relative to the one before. 0 maintains peak height
              #defaults to 0

traj = 'pulse' #you have 3 choices here and they are all strings: pulse, pulse_return, pulse_return_loop
          #pulse simply creates the trajectory from A to B
          #pulse_return creates the trajectory from start to finish then returns it to the start
          #pulse_return 

#you can create your custom parametrized time function here and pass it into the trajectory builder
#the properties are listed above

#ONLY ADD THE TIME FUNCTION TO THE BUILDER IF YOU WANT TO. The commands simply pack the parameters and calls the trajectory builder
var_list = ['return_size', 'pulse_frac','tau','pulse_size','n_loops','return_scale','peak_scale']
param_list = param_builder(var_list)
############################################################################################################

t = [t_start,t_end]
t9 = np.array([t9_1,t9_2,t9_3])
nn = np.array([nn_1,nn_2,nn_3])
x_t,y_t,time = traj_builder(t,t9,nn,traj,param_list) #if you have a custom parametrization for the angle as a function of time,
                                                     #pass it after param_list


Throughout the examples, the red stars are the original 3 points defined by the user.

For the 1st example, we create a simple trajectory that goes from start till end passing through the peak

In [ ]:
plt.scatter(x_t,10**y_t)
plt.scatter(t9,10**nn,marker='*',color = 'red')
plt.yscale('log')
plt.show()

Using the same parameters, let's create a trajectory that returns back to the original point

In [ ]:
traj = 'pulse_return'
x_t,y_t,time = traj_builder(t,t9,nn,traj,param_list)

plt.scatter(x_t,10**y_t)
plt.scatter(t9,10**nn,marker='*',color = 'red')
plt.yscale('log')
plt.show()


Now let's do a sequence of pulses where the starting point shifts in temperature 50% of the starting, now that we're looping, we have to adjust $\tau$ and repack the parameter list. We use 50%, which is an unrealistic shift just to exaggerate the difference in the loops

In [ ]:
traj = 'pulse_return_loop'

tau = 1e+11
param_list = param_builder(var_list)
x_t,y_t,time = traj_builder(t,t9,nn,traj,param_list)

plt.scatter(x_t,10**y_t)
plt.scatter(t9,10**nn,marker='*',color = 'red')
plt.yscale('log')
plt.show()


Now, let's animate how the trajectory evolves in time for the last trajectory that was generated. The black star will indicate where the system is at a given point in time.

In [ ]:
fig = plt.figure()
def update_fig(i):
    fig.clear()
    fig.suptitle(r'time(s): %8.2e' %(time[i]))
    
    plt.scatter(x_t,10**y_t)
    plt.scatter(x_t[i],10**y_t[i],marker = 'x',color = 'black')
    plt.yscale('log')
    plt.draw()

anim = animation.FuncAnimation(fig,update_fig,len(time))
display(HTML(anim.to_jshtml()))
plt.close()